Multi-task UnitedNet with PBMC dataset (peturbation of control cells with IF)

Both of the classification and the translation network's parameters are taken into account in the multi-task way orchestrated by UnitedNet.

In [1]:
# prepare dataset
# dataset from https://github.com/theislab/scgen-reproducibility, already preprocessed

import scanpy as sc

train_data = "data/pbmc/train_pbmc.h5ad"
valid_data = "data/pbmc/valid_pbmc.h5ad"

# pbmc data from scGen is already split into train and valid, but we will concatenate them to do a custom split
sc_data_train = sc.read_h5ad(train_data)
sc_data_valid = sc.read_h5ad(valid_data)
pbmc = sc_data_train.concatenate(sc_data_valid)

In [8]:
pbmc

AnnData object with n_obs × n_vars = 18868 × 6998
    obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type', 'batch'
    var: 'gene_symbol', 'n_cells'
    obsm: 'X_pca', 'X_tsne', 'X_umap'

In [9]:
pbmc.obs.groupby(['cell_type', 'condition']).size()

/tmp/ipykernel_3453047/191267336.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pbmc.obs.groupby(['cell_type', 'condition']).size()


cell_type    condition 
CD4T         control       2715
             stimulated    3483
CD14+Mono    control       2184
             stimulated     698
B            control        928
             stimulated    1105
CD8T         control        643
             stimulated     594
NK           control        571
             stimulated     733
FCGR3A+Mono  control       1232
             stimulated    2790
Dendritic    control        670
             stimulated     522
dtype: int64

In [10]:
pbmc.obs.groupby(['condition']).size()

/tmp/ipykernel_3453047/225387844.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pbmc.obs.groupby(['condition']).size()


condition
control       8943
stimulated    9925
dtype: int64

In [11]:
# simple inspection

# sc.pp.pca(pbmc)
# sc.pp.neighbors(pbmc)
# sc.tl.tsne(pbmc)
# sc.pl.tsne(pbmc, color=['condition', 'cell_type'], legend_loc='on data', legend_fontsize='small')

In [12]:
from scButterfly.split_datasets import unpaired_split_dataset_perturb


# create pairs of data using scButterfly technique
control_data = pbmc[pbmc.obs.condition == 'control']
stimulate_data = pbmc[pbmc.obs.condition == 'stimulated']

control_data.obs.index = [str(i) for i in range(control_data.X.shape[0])]
stimulate_data.obs.index = [str(i) for i in range(stimulate_data.X.shape[0])]

cell_type_list = list(control_data.obs.cell_type.cat.categories)

In [13]:
id_list, id_list_dict = unpaired_split_dataset_perturb(control_data, stimulate_data)

optimal transport array torch.Size([2715, 3483])
CD4T, control num 2715 stimulate num 3483

optimal transport array torch.Size([2184, 698])
CD14+Mono, control num 2184 stimulate num 698



In [8]:
#print(len(id_list[0]))
for idx, cell_type in enumerate(cell_type_list):
    print(cell_type)
    train_id_control, train_id_peturb, validation_id_control, validation_id_peturb, test_id_control, test_id_peturb = id_list[idx]

    control = len(train_id_control) + len(validation_id_control) + len(test_id_control)
    peturb = len(train_id_peturb) + len(validation_id_peturb) + len(test_id_peturb) 
    control_dict = id_list_dict[cell_type]["control"]
    peturb_dict = id_list_dict[cell_type]["stimulated"]
    print("control", len(train_id_control), len(validation_id_control), len(test_id_control), control)
    print("control", len(control_dict["train"]), len(control_dict["validation"]), len(control_dict["test"]))
    print("peturb", len(train_id_peturb), len(validation_id_peturb), len(test_id_peturb), peturb)    
    print("peturb", len(peturb_dict["train"]), len(peturb_dict["validation"]), len(peturb_dict["test"]))
    print()
    
# Some notes due to the lack of code readability from scButterfly:
# - the rows of the id_list corresponds to the cell types, but they don't have any meaning.
# - the rows actually correspond to k-fold cross validation. One batch is a group of a cell type that is held out.
# - the cell_type of the row and the one that is held out are irrelevant.
# - for the test, the control and the stimulated can have different size. But for train and validation we need pairs.

CD4T
control 6616 1657 670 8943
control 6616 1657 670
peturb 6616 1657 522 8795
peturb 6616 1657 522

CD14+Mono
control 6410 1605 928 8943
control 6410 1605 928
peturb 6410 1605 1105 9120
peturb 6410 1605 1105

B
control 6167 1544 1232 8943
control 6167 1544 1232
peturb 6167 1544 2790 10501
peturb 6167 1544 2790

CD8T
control 5405 1354 2184 8943
control 5405 1354 2184
peturb 5405 1354 698 7457
peturb 5405 1354 698

NK
control 4980 1248 2715 8943
control 4980 1248 2715
peturb 4980 1248 3483 9711
peturb 4980 1248 3483

FCGR3A+Mono
control 6696 1676 571 8943
control 6696 1676 571
peturb 6696 1676 733 9105
peturb 6696 1676 733

Dendritic
control 6638 1662 643 8943
control 6638 1662 643
peturb 6638 1662 594 8894
peturb 6638 1662 594



In [20]:
# create adatas for pair of control and stimulated

batch = cell_type_list[0]
id_list_batch = id_list_dict[batch]
control = id_list_batch["control"]
peturb = id_list_batch["stimulated"]
train_id_control = control["train"]
validation_id_control = control["validation"]
test_id_control = control["test"]
train_id_peturb = peturb["train"]
validation_id_peturb = peturb["validation"]
test_id_peturb = peturb["test"]

control_train = pbmc[train_id_control]
control_valid = pbmc[validation_id_control]
control_test = pbmc[test_id_control]

peturb_train = pbmc[train_id_peturb]
peturb_valid = pbmc[validation_id_peturb]
peturb_test = pbmc[test_id_peturb]

all = [control_train, control_valid, control_test, peturb_train, peturb_valid, peturb_test]
for adata in all:
    adata.obs["label"] = adata.obs["cell_type"]
    adata.X = adata.X.toarray()

adatas_train = [control_train, peturb_train]
adatas_valid = [control_valid, peturb_valid]
adatas_test = [control_test, peturb_test]

adatas_train
features_num = adatas_train[0].n_vars

6998


In [21]:
from unitednet.interface import UnitedNet


pbmc_config = {
    "train_batch_size": 512,
    "finetune_batch_size": 5000,
    "transfer_batch_size": 512,
    "train_epochs": 10,
    "finetune_epochs": 10,
    "transfer_epochs": 20,
    "train_task": "supervised_group_identification", # -> translation, classification
    "finetune_task": None,
    "transfer_task": None,
    "train_loss_weight": None,
    "finetune_loss_weight": None,
    "transfer_loss_weight": None,
    "lr": 0.01,
    "checkpoint": 1,
    "n_head": 1,
    "noise_level":[0,0],
    "fuser_type":"WeightedMean",
    "encoders": [
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": True,
        },
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
    ],
    "latent_projector": None,
    "decoders": [
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "sigmoid"],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", None],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
    ],
    "discriminators": [
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
    ],
    "projectors": {
        "input": 64,
        "hiddens": [],
        "output": 100,
        "use_biases": [True],
        "dropouts": [0],
        "activations": ["relu"],
        "use_batch_norms": [False],
        "use_layer_norms": [True],
    },
    "clusters": {
        "input": 100,
        "hiddens": [],
        "output": len(cell_type_list),
        "use_biases": [False],
        "dropouts": [0],
        "activations": [None],
        "use_batch_norms": [False],
        "use_layer_norms": [False],
    },
}

device="cuda:0"
test_batch= batch
root_save_path = f"saved_results/thesis/pbmc/multi_task/{test_batch}"

model = UnitedNet(f"{root_save_path}/{test_batch}", device=device, technique=pbmc_config)

In [1]:
model.train(adatas_train, verbose=True)
model.finetune(adatas_train, verbose=True)
#model.transfer(adatas_train, adatas_transfer = adatas_test, verbose=True)
#print(model.evaluate(adatas_test))
# evaluation should be different for peturbation

NameError: name 'model' is not defined

Single-task translation 

The classification network's parameters are ignored.

In [ ]:
pbmc_config = {
    "train_batch_size": 512,
    "finetune_batch_size": 5000,
    "transfer_batch_size": 512,
    "train_epochs": 10,
    "finetune_epochs": 10,
    "transfer_epochs": 20,
    "train_task": "cross_model_prediction", # -> translation
    "finetune_task": None,
    "transfer_task": None,
    "train_loss_weight": None,
    "finetune_loss_weight": None,
    "transfer_loss_weight": None,
    "lr": 0.01,
    "checkpoint": 1,
    "n_head": 1,
    "noise_level":[0,0],
    "fuser_type":"WeightedMean",
    "encoders": [
        {
            "input": 13634,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": True,
        },
        {
            "input": 4000,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
    ],
    "latent_projector": None,
    "decoders": [
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": 13634,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "sigmoid"],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": 4000,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", None],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
    ],
    "discriminators": [
        {
            "input": 13634,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
        {
            "input": 4000,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
    ],
    "projectors": {
        "input": 64,
        "hiddens": [],
        "output": 100,
        "use_biases": [True],
        "dropouts": [0],
        "activations": ["relu"],
        "use_batch_norms": [False],
        "use_layer_norms": [True],
    },
    "clusters": {
        "input": 100,
        "hiddens": [],
        "output": 22,
        "use_biases": [False],
        "dropouts": [0],
        "activations": [None],
        "use_batch_norms": [False],
        "use_layer_norms": [False],
    },
}

device="cuda:0"
test_batch="batch1"
root_save_path = f"saved_results/thesis/pbmc/translation/{test_batch}"

model = UnitedNet(f"{root_save_path}/{test_batch}", device=device, technique=pbmc_config)

Single-task cell type annotation

The translation networks' parameters are ignored.

In [ ]:
pbmc_config = {
    "train_batch_size": 512,
    "finetune_batch_size": 5000,
    "transfer_batch_size": 512,
    "train_epochs": 10,
    "finetune_epochs": 10,
    "transfer_epochs": 20,
    "train_task": "supervised_group_identification_only", # -> classification
    "finetune_task": None,
    "transfer_task": None,
    "train_loss_weight": None,
    "finetune_loss_weight": None,
    "transfer_loss_weight": None,
    "lr": 0.01,
    "checkpoint": 1,
    "n_head": 1,
    "noise_level":[0,0],
    "fuser_type":"WeightedMean",
    "encoders": [
        {
            "input": 13634,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": True,
        },
        {
            "input": 4000,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
    ],
    "latent_projector": None,
    "decoders": [
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": 13634,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "sigmoid"],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": 4000,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", None],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
    ],
    "discriminators": [
        {
            "input": 13634,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
        {
            "input": 4000,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
    ],
    "projectors": {
        "input": 64,
        "hiddens": [],
        "output": 100,
        "use_biases": [True],
        "dropouts": [0],
        "activations": ["relu"],
        "use_batch_norms": [False],
        "use_layer_norms": [True],
    },
    "clusters": {
        "input": 100,
        "hiddens": [],
        "output": 22,
        "use_biases": [False],
        "dropouts": [0],
        "activations": [None],
        "use_batch_norms": [False],
        "use_layer_norms": [False],
    },
}

device="cuda:0"
test_batch="batch1"
root_save_path = f"saved_results/thesis/pbmc/classification/{test_batch}"

model = UnitedNet(f"{root_save_path}/{test_batch}", device=device, technique=pbmc_config)